In [22]:
import pandas as pd
import plotly.graph_objects as go
import utils


In [23]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))


In [24]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [25]:
df_raw.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [26]:
df = df_raw[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_c', 'ask_c']].copy()

In [27]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c
0,2020-01-01T22:00:00.000000000Z,108.651,108.761,108.607,108.673,108.667,108.679
1,2020-01-02T02:00:00.000000000Z,108.671,108.758,108.658,108.742,108.736,108.747
2,2020-01-02T06:00:00.000000000Z,108.744,108.842,108.696,108.839,108.833,108.845
3,2020-01-02T10:00:00.000000000Z,108.836,108.866,108.658,108.694,108.688,108.700
4,2020-01-02T14:00:00.000000000Z,108.697,108.748,108.212,108.529,108.523,108.535


In [28]:
df_raw.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [29]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c
0,2020-01-01T22:00:00.000000000Z,108.651,108.761,108.607,108.673,108.667,108.679
1,2020-01-02T02:00:00.000000000Z,108.671,108.758,108.658,108.742,108.736,108.747
2,2020-01-02T06:00:00.000000000Z,108.744,108.842,108.696,108.839,108.833,108.845
3,2020-01-02T10:00:00.000000000Z,108.836,108.866,108.658,108.694,108.688,108.700
4,2020-01-02T14:00:00.000000000Z,108.697,108.748,108.212,108.529,108.523,108.535


In [36]:
def direction(row):
    if row.mid_c > row.mid_o:
        return 1
    return -1

def get_signal(row):
    if row.mid_h_prev > row.mid_h and row.mid_l_prev < row.mid_l:
        return row.DIRECTION_prev
    return 0

In [39]:
df = df_raw[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_c', 'ask_c']].copy()
df['RANGE'] = df.mid_h - df.mid_l
df['mid_h_prev'] = df.mid_h.shift(1)
df['mid_l_prev'] = df.mid_l.shift(1)
df['RANGE_prev'] = df.RANGE.shift(1)
df['DIRECTION'] = df.apply(direction, axis=1)
df['DIRECTION_prev'] = df.DIRECTION.shift(1).fillna(0).astype(int)
df.dropna(inplace=True)
df['SIGNAL'] = df.apply(get_signal, axis=1)
df.reset_index(drop=True, inplace=True)


In [40]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c,RANGE,mid_h_prev,mid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL
0,2020-01-02T02:00:00.000000000Z,108.671,108.758,108.658,108.742,108.736,108.747,0.100,108.761,108.607,0.154,1,1,1
1,2020-01-02T06:00:00.000000000Z,108.744,108.842,108.696,108.839,108.833,108.845,0.146,108.758,108.658,0.100,1,1,0
2,2020-01-02T10:00:00.000000000Z,108.836,108.866,108.658,108.694,108.688,108.700,0.208,108.842,108.696,0.146,-1,1,0
3,2020-01-02T14:00:00.000000000Z,108.697,108.748,108.212,108.529,108.523,108.535,0.536,108.866,108.658,0.208,-1,-1,0
4,2020-01-02T18:00:00.000000000Z,108.526,108.572,108.494,108.561,108.511,108.611,0.078,108.748,108.212,0.536,1,-1,-1


In [42]:
df[df.SIGNAL!=0].shape

(804, 14)

In [43]:
df.shape

(4674, 14)